Welcome to one of the collaborative Jupyter/Spark environments in ZHAW. You are not yet connected to Sparky by default. However, the necessary code template makes this a quick process. Keep in mind that you are sharing both the Jupyter environment and the Sparky cluster with others. Custom Python packages on the notebook/Spark driver side are installed with %pip install.

# Preprocessing

In [1]:
zhawaccount = "tsianthe"  # TODO set this to your ZHAW-Kürzel

In [2]:
import sparky
import pyspark
#import slash
import pyspark.sql
sc = sparky.connect(f"sparknotebook-{zhawaccount}", 2)
spark = pyspark.sql.SparkSession.builder.getOrCreate()

~~~ Sparky module loaded ~~~


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/24 13:31:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/06/24 13:31:11 WARN Utils: Service 'sparkDriver' could not bind on port 5555. Attempting port 5556.
24/06/24 13:31:11 WARN Utils: Service 'org.apache.spark.network.netty.NettyBlockTransferService' could not bind on port 4444. Attempting port 4445.


Attached to Sparky cluster context from sparky-collab as sparknotebook-tsianthe.
Requested 2 cores; real number might be less.


In [3]:
!python --version

Python 3.11.4


In [4]:
import time

In [5]:
# Read csv-files
read_time = time.time()
df_customer = spark.read.format("csv").option("header", "true").load("Customermaster.csv")
df_items = spark.read.format("csv").option("header", "true").load("Itemmaster.csv")
df_orders = spark.read.format("csv").option("header", "true").load("Orderlines.csv")
df_currate = spark.read.format("csv").option("header", "true").load("ExchangeRates.csv")
read_time_finish = time.time()
print(f"Time taken to read: {read_time_finish - read_time:.2f} seconds")

Time taken to read: 6.41 seconds


In [6]:
# Clean Data
from pyspark.sql.functions import trim, col, to_date, when

# Customer ---------------------------
df_customer = df_customer.withColumn("Status", col("Status").cast("integer"))
df_customer = df_customer.withColumn("Potential", col("Potential").cast("float"))
df_customer = df_customer.withColumn("Entry Date", to_date(col("Entry Date"), "yyyyMMdd"))

# Item ---------------------------
df_items = df_items.withColumn("Item number", trim(df_items["Item number"]))
df_items = df_items.withColumn("Itemgroup", trim(df_items["Itemgroup"]))
df_items = df_items.withColumn("Itemgroup", col("Itemgroup").cast("integer"))
df_items = df_items.withColumn("Productgroup", trim(df_items["Productgroup"]))
df_items = df_items.withColumn("Itemtype", trim(df_items["Itemtype"]))

# Define mapping dictionary for renaming
rename_map = {
    "1000": "Valve",
    "1100": "Actuator",
    "1200": "Fitting",
    "1300": "Instrumentation",
    "1400": "Air Line Equipment",
    "1500": "Vacuum Unit",
    "1600": "Heat Exchanger",
    "1700": "Hydraulics",
    "1800": "Filter",
    "1900": "Other",
    "": "Other",
    "2000": "3rdParty"
}

# Rename values in the "Itemgroup" column based on the mapping dictionary
#df_items = df_items.withColumn("Itemgroup", 
#                    when(col("Itemgroup").isin(list(rename_map.keys())), 
#                         rename_map[col("Itemgroup")]).otherwise(col("Itemgroup")))

# Order ---------------------------
df_orders = df_orders.withColumn("Customernumber", trim(df_orders["Customernumber"]))
df_orders = df_orders.withColumn("Item number", trim(df_orders["Item number"]))
df_orders = df_orders.withColumn("Status", col("Status").cast("integer"))
df_orders = df_orders.withColumn("Net price", col("Net price").cast("float"))

quantity_columns = [
    "Ordered quantity", "Confirmed quantity", "Remaining quantity", 
    "Allocated quantity", "Picking list quantity", 
    "Delivered quantity", "Invoiced quantity"
]

for column in quantity_columns:
    df_orders = df_orders.withColumn(column, col(column).cast("integer"))
    
date_columns = [
    "Requested delivery date", "Confirmed delivery date", 
    "Departure date", "Planning date", "Registration date"
]

for column in date_columns:
    df_orders = df_orders.withColumn(column, to_date(col(column), "yyyyMMdd"))

# Exchange Rate ---------------------------
df_currate = df_currate.withColumn("ExchangeRate", col("ExchangeRate").cast("float"))
df_currate = df_currate.withColumn("ActiveDate", to_date(col("ActiveDate"), "yyyyMMdd"))

In [7]:
# Store the files in Parquet format 
df_customer.write.mode("overwrite").parquet("./customermaster.parquet")
df_items.write.mode("overwrite").parquet("./itemmaster.parquet")
df_orders.write.mode("overwrite").parquet("./orderlines.parquet")
df_currate.write.mode("overwrite").parquet("./exchangerates.parquet")

In [8]:
# Read parquet-files
read_time_start = time.time()
df_customer_p = spark.read.parquet("./customermaster.parquet", header=True, inferSchema=True)
df_items_p = spark.read.parquet("./itemmaster.parquet", header=True, inferSchema=True)
df_orders_p = spark.read.parquet("./orderlines.parquet", header=True, inferSchema=True)
df_currate_p = spark.read.parquet("./exchangerates.parquet", header=True, inferSchema=True)
read_time_finish = time.time()
print(f"Time taken to read: {read_time_finish - read_time_start:.2f} seconds")

Time taken to read: 0.86 seconds


In [9]:
# Return the number of records per file.
record_count = df_customer_p.count()
print(f"Number of records in customer: {record_count}\n")
record_count = df_items_p.count()
print(f"Number of records in items: {record_count}\n")
record_count = df_orders_p.count()
print(f"Number of records in orders: {record_count}\n")

Number of records in customer: 39250

Number of records in items: 2226456

Number of records in orders: 465209



In [10]:
df_customer_p.head()

Row(Customernumber='K072870000', Status=10, CGC='XXX', Industry='M', Potential=2000.0, DiscountModel='SMCSTDCHFA', PriceList='STDPRICE  ', Currency='CHF', TargetFlag=' ', IndirectFlag=' ', Oldcustomernr='          ', Newcustomernr='K01286JIT0', Incoterms='EXW', CRMflag='5', CRMID='6dcfddfa-1c83-11ee-98f4-0050568cbcf8', SPC='KM03', Entry Date=datetime.date(2023, 7, 6))

In [11]:
df_items_p.head()

Row(Item number='02419283', Status='20', Itemgroup=1900, Productgroup='999', Itemtype='2', EntryDate='20220412', Responsible='LOCAL ES  ')

In [12]:
df_orders_p.head()

Row(Company='100', Division='CH0', Customernumber='K022520000', CustomerOrderNumber='0100832280', Linenumber='1', LineSuffix='0', Status=5, Facility='K01', Warehouse='K01', Item number='01741721', Ordered quantity=1, Confirmed quantity=0, Remaining quantity=1, Allocated quantity=0, Picking list quantity=0, Delivered quantity=0, Invoiced quantity=0, Net price=396.7223205566406, Requested delivery date=datetime.date(2022, 4, 4), Confirmed delivery date=datetime.date(2022, 4, 4), Departure date=datetime.date(2022, 4, 4), Planning date=datetime.date(2022, 4, 4), Registration date=datetime.date(2022, 3, 29))

In [13]:
df_currate_p.tail(5)

[Row(ActiveDate=datetime.date(2021, 9, 30), LocalCurrency='CHF', ForeignCurrency='USD', ExchangeRate=0.8956999778747559),
 Row(ActiveDate=datetime.date(2022, 3, 31), LocalCurrency='CHF', ForeignCurrency='USD', ExchangeRate=0.8956999778747559),
 Row(ActiveDate=datetime.date(2020, 3, 24), LocalCurrency='CHF', ForeignCurrency='USD', ExchangeRate=0.9300000071525574),
 Row(ActiveDate=datetime.date(2021, 7, 29), LocalCurrency='CHF', ForeignCurrency='ZAR', ExchangeRate=0.06333500146865845),
 Row(ActiveDate=datetime.date(2021, 11, 4), LocalCurrency='CHF', ForeignCurrency='ZAR', ExchangeRate=0.05507500097155571)]

In [14]:
df_customer_p.createOrReplaceTempView("customer")
df_items_p.createOrReplaceTempView("items")
df_orders_p.createOrReplaceTempView("orders")
df_currate_p.createOrReplaceTempView("exchangerate")
df_customer.createOrReplaceTempView("customerCSV")
df_items.createOrReplaceTempView("itemsCSV")
df_orders.createOrReplaceTempView("ordersCSV")
df_currate.createOrReplaceTempView("exchangerateCSV")

In [15]:
# Count unique sold Items - does not work?
read_time_start = time.time()
result = spark.sql("""Select Count(Distinct `Item number`) as UniqueItems From orders""")
read_time_finish = time.time()
result.show()
print(f"Time taken to read: {read_time_finish - read_time_start:.2f} seconds")


+-----------+
|UniqueItems|
+-----------+
|      39231|
+-----------+

Time taken to read: 0.11 seconds


In [16]:
# Count Orderlines in CHF in 2021 in CHF
read_time_start = time.time()
result = spark.sql("""
SELECT
    Count(or.`Customernumber`) as CountOrderLines
FROM 
    orders as or
LEFT JOIN 
    customer as cu
ON
    or.Customernumber = cu.Customernumber
WHERE
    cu.Currency = 'CHF' AND
    or.Status > 5 and or.Status < 90 AND
    or.`Registration date` BETWEEN '2021-01-01' AND '2021-12-31'
""")
read_time_finish = time.time()
result.show()
print(f"Time taken to read: {read_time_finish - read_time_start:.2f} seconds")

+---------------+
|CountOrderLines|
+---------------+
|          93069|
+---------------+

Time taken to read: 0.08 seconds


In [17]:
# Overall Orderamount in 2021 with CHF-Customers
read_time_start = time.time()
result = spark.sql("""
SELECT
    SUM(`Ordered quantity` * `Net price`)/1000000 as MioOrderAmountCHF
FROM 
    orders as or
LEFT JOIN 
    customer as cu
ON
    or.Customernumber = cu.Customernumber
WHERE
    cu.Currency = 'CHF' AND
    or.Status > 5 AND or.Status < 90 AND
    or.`Registration date` BETWEEN '2021-01-01' AND '2021-12-31' """)
read_time_finish = time.time()
result.show()
print(f"Time taken to read: {read_time_finish - read_time_start:.2f} seconds")

+------------------+
| MioOrderAmountCHF|
+------------------+
|19.785846694221533|
+------------------+

Time taken to read: 0.05 seconds


In [18]:
# Select only latest Exchange-Rates due to ranking
read_time_start = time.time()
result = spark.sql("""
Select *, 
dense_rank() over (Partition by ForeignCurrency Order By ActiveDate desc) as rankno
From exchangerate
where ForeignCurrency != 'CHF' """)
read_time_finish = time.time()
result.show()
print(f"Time taken to read: {read_time_finish - read_time_start:.2f} seconds")

+----------+-------------+---------------+------------+------+
|ActiveDate|LocalCurrency|ForeignCurrency|ExchangeRate|rankno|
+----------+-------------+---------------+------------+------+
|2021-07-29|          CHF|            CZK|    0.043715|     1|
|2022-05-31|          CHF|            DKK|     0.13163|     1|
|2022-04-30|          CHF|            DKK|    0.131223|     2|
|2022-03-31|          CHF|            DKK|    0.130946|     3|
|2022-03-31|          CHF|            DKK|    0.130219|     3|
|2022-01-31|          CHF|            DKK|    0.125392|     4|
|2021-12-31|          CHF|            DKK|    0.124247|     5|
|2021-11-30|          CHF|            DKK|    0.128267|     6|
|2021-10-31|          CHF|            DKK|    0.128699|     7|
|2021-09-30|          CHF|            DKK|    0.129661|     8|
|2021-09-30|          CHF|            DKK|    0.130219|     8|
|2021-08-31|          CHF|            DKK|    0.128605|     9|
|2021-07-31|          CHF|            DKK|    0.129064|

In [19]:
# Overall Orderamount in 2021 with all-Customers and latest Exchange Rate
read_time_start = time.time()
result = spark.sql("""
SELECT
    SUM(or.`Ordered quantity` * or.`Net price` * ra.ExchangeRate)/1000000 as MioOrderAmountCHF
FROM 
    orders as or
LEFT JOIN 
    customer as cu
ON
    or.Customernumber = cu.Customernumber
LEFT JOIN 
    (SELECT * 
        FROM(
        SELECT
            ActiveDate,
            ForeignCurrency,
            ExchangeRate,
            dense_rank() over (Partition by ForeignCurrency Order By ActiveDate desc) as rankno
        From 
            exchangerate) as exchrate
    WHERE 
        exchrate.rankno = 1) as ra
ON
    cu.Currency = ra.ForeignCurrency
WHERE
    or.Status > 5 AND or.Status < 90 AND
    or.`Registration date` BETWEEN '2021-01-01' AND '2021-12-31' """)
read_time_finish = time.time()
result.show()
print(f"Time taken to read: {read_time_finish - read_time_start:.2f} seconds")

+------------------+
| MioOrderAmountCHF|
+------------------+
|30.322033072304457|
+------------------+

Time taken to read: 0.05 seconds


In [20]:
# Overall Orderamount in 2021 with all-Customers and latest Exchange Rate by Itemgroup
read_time_start = time.time()
result = spark.sql("""
SELECT
    it.`Itemgroup`,
    SUM(or.`Ordered quantity` * or.`Net price` * COALESCE(ra.ExchangeRate, 1)) / 1000000 as MioOrderAmountCHF
FROM 
    orders as or
LEFT JOIN 
    customer as cu
ON
    or.Customernumber = cu.Customernumber
LEFT JOIN 
    (SELECT
        ForeignCurrency,
        ExchangeRate,
        dense_rank() over (Partition by ForeignCurrency Order By ActiveDate desc) as rankno
     FROM 
        exchangerate
    ) AS ra
ON
    cu.Currency = ra.ForeignCurrency AND ra.rankno = 1
LEFT JOIN
    items as it
ON
    or.`Item number` = it.`Item number`
WHERE
    or.Status > 5 AND or.Status < 90 AND
    or.`Registration date` BETWEEN '2021-01-01' AND '2021-12-31'
GROUP BY
    it.`Itemgroup`""")
read_time_finish = time.time()
result.show()
print(f"Time taken to read: {read_time_finish - read_time_start:.2f} seconds")

+---------+--------------------+
|Itemgroup|   MioOrderAmountCHF|
+---------+--------------------+
|     1700| 0.02216495757484436|
|     1500|  0.9129024241651148|
|     1300|  1.1374349568749667|
|     1400|   3.483150809136237|
|     1100|   6.784189984872997|
|     1800|0.030853060820102692|
|     1600|  1.6795161772317886|
|     1000|   6.633493100238214|
|     1200|  3.3430149478758806|
|     2000|6.037993450164795E-4|
|     1900|   6.294708854169264|
+---------+--------------------+

Time taken to read: 0.07 seconds


In [ ]:
sc.stop()